In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
sys.path.append('../experiments')
import seaborn as sns
import os
import pandas as pd
from copy import deepcopy
from matplotlib import pyplot as plt
from os.path import join
import numpy as np
import imodelsx.process_results
from neuro.features import qa_questions, feature_spaces
from neuro.data import story_names
from neuro.features.stim_utils import load_story_wordseqs, load_story_wordseqs_huge
import random
import json
import joblib
from tqdm import tqdm
from collections import defaultdict
fit_encoding = __import__('02_fit_encoding')

### Get data

In [2]:
# story_names_list = story_names.get_story_names(
# 'shared', 'train', use_huge=True)
story_names_list = story_names.get_story_names(
    all=True)
print('loaded', len(story_names_list), 'stories')
# wordseqs = feature_spaces.get_story_wordseqs(story_names_train)
wordseqs = load_story_wordseqs_huge(story_names_list)

ngrams_list_total = []
for story in story_names_list:
    ngrams_list = feature_spaces.get_ngrams_list_main(
        wordseqs[story], num_ngrams_context=10)
    # print(ngrams_list[:10])
    ngrams_list_total.extend(ngrams_list)

questions = qa_questions.get_questions('v3_boostexamples', full=True)

prompt_template = 'Read the input then answer a question about the input.\nInput: {example}\nQuestion: {question} Answer yes or no.'
prompt = prompt_template.format(
    example=ngrams_list[100], question='Does the sentence include a metaphor?')
# print(prompt_template.format(example=ngrams_list[100], question=questions[0]))
print(prompt)

/home/chansingh/fmri/neuro/data/story_names.py:72: UserWarning: Loading all stories, ignoring subject / train_or_test
  warnings.warn('Loading all stories, ignoring subject / train_or_test')


loaded 103 stories
Read the input then answer a question about the input.
Input: and tetris has all of them your job is to fit
Question: Does the sentence include a metaphor? Answer yes or no.


In [4]:
len(ngrams_list_total)

195190

In [ ]:
len(questions)

In [ ]:
rng = np.random.RandomState(42)
# select 100 random examples
ngrams_list_sample = rng.choice(ngrams_list_total, 100, replace=False)

In [ ]:
ngrams_list_sample

# Estimate cost for running gpt-4 on all of these...

In [ ]:
# lm = imodelsx.llm.get_llm('gpt-4')
lm = imodelsx.llm.get_llm('gpt-4-turbo-0125-spot')

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

In [ ]:
resp = lm(messages, max_new_tokens=1, temperature=0, return_str=False)

In [ ]:
input_tokens = len(ngrams_list_total) * 56
print(len(ngrams_list_total), 'inputs per question')
print(f'{input_tokens:,} tokens per question')
print(f'{100 * input_tokens:,} tokens for 100 questions')

In [54]:
from ratelimit import limits, sleep_and_retry
import requests


@sleep_and_retry
@limits(calls=50, period=10)  # in secs
def call_api_limited(messages):
    resp = lm(messages, max_new_tokens=1, temperature=0,
              return_str=False, verbose=False)
    return resp


def call_api(messages):
    resp = lm(messages, max_new_tokens=1, temperature=0,
              return_str=False, verbose=False)
    return resp


answers = []
for i, ngrams in enumerate(ngrams_list_sample[:3]):
    out_file = f'/home/chansingh/mntv1/deep-fMRI/qa/cache_gpt/ngram_{i}.pkl'
    answers = []
    if not os.path.exists(out_file):
        for question in tqdm(questions):
            prompt = prompt_template.format(
                example=ngrams, question=question)
            messages = [
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]
            # print(messages)
            already_cached = lm(messages, max_new_tokens=1, temperature=0,
                                return_str=False, return_false_if_not_cached=True, verbose=False)
            if already_cached:
                resp = call_api(messages)
            else:
                resp = call_api_limited(messages)
            # print(i, resp.choices[0].message.content)
            answers.append(resp.choices[0].message.content)

        answers = pd.Series(answers).str.lower()
        assert set(answers.values) == {'yes', 'no'}
        joblib.dump((answers.values == 'yes').astype(bool), out_file)

 58%|█████▊    | 394/674 [01:12<01:06,  4.20it/s]